In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
# one needs to import those packages which are needed; best to be done at the beginning of the program.
import networkx.algorithms.community as nx_comm
import numpy as np
import pandas as pd
import scipy as sp
import scipy.sparse as ss
import random as rn
from heapq import nlargest
from collections import Counter
from itertools import combinations

# some basic settings for plotting figures
import matplotlib.pyplot as plt
%matplotlib inline 
font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 10}

plt.rc('font', **font)
import numpy.linalg as LA
from sklearn.cluster import KMeans

In [2]:
from bokeh.io import output_notebook, show, save

In [3]:
G0 = nx.read_weighted_edgelist("4932.protein.links.v11.5.txt",comments="#",nodetype=str)

In [4]:
threshold_score = 700
for edge in G0.edges: 
    weight = list(G0.get_edge_data(edge[0],edge[1]).values())
    if(weight[0] <= threshold_score):
        G0.remove_edge(edge[0],edge[1])

In [5]:
# remove the essential nodes from G0
ess=pd.read_csv("essential_pro.csv",header=None)
ess_pro=pd.Series.to_list(ess[1])
for i in range(len(ess_pro)):
    ess_pro[i]='4932.'+ess_pro[i]
G0.remove_nodes_from(ess_pro)

In [6]:
# narrow our selection to the proteins connected to ours
nodes = nx.shortest_path(G0,'4932.YKL126W').keys()
G=G0.subgraph(nodes)

In [7]:
# time to define a parent class of network
class Network:
    R = 1
    N = 10
    MIN_SIZE = 4

    def __init__(self, graph, homologue='4932.YKL126W', partition_method="nx_louvain",partitions=[]):
        self.graph = graph
        self.homologue = homologue
        self.partition_method = partition_method

        self._partitions = partitions
        self.homologue_communities = []
        self.homologue_members={}
        self.central_nodes = []
        self.important_nodes = {}
        self.homologue_index=[]
        self.community_neighbours=[]
        self.adjacent_communities = []
        self.central_nodes_neighbour = [] 
        self.important_nodes_neighbour = {}

        if self.partitions == []:
            self.set_partitions_robust()
        self.set_homologue_communities()
        self.set_central_nodes_robust()
        self.set_important_nodes()

    def set_partitions_robust(self):
        def find_partition(graph, partition_method, s):
            if partition_method == "nx_louvain":
                return nx_comm.louvain_communities(graph, resolution=Network.R, seed=s)

            if partition_method == "other_louvain":
                # some kind of community collection
                return None

        for i in range(Network.N):
            partition = find_partition(self.graph, self.partition_method, i)
            self.partitions.append(partition)

    def set_homologue_communities(self):
        for part in self.partitions:
            for i in range(len(part)):
                if self.homologue in part[i]:
                    sub = self.graph.subgraph(part[i])
                    self.homologue_communities.append(sub)
                    self.homologue_index.append(i)
                    break
    
    def count_homologue_comm_members(self):
        get_subgraph_nodes = lambda x: self.graph.subgraph(x).nodes
        homo_networks = map(get_subgraph_nodes, self.homologue_communities)
        # count the number of subgraph each node occurs in
        flat_comm_nodes = [y for x in homo_networks for y in x]
        for node in list(set(flat_comm_nodes)):
            self.homologue_members[node] = flat_comm_nodes.count(node)

    def set_central_nodes_robust(self):
        def find_central_nodes(community,n=5):
            """return a list of the most significant nodes
            according to three centrality measures"""
            deg = nx.degree_centrality(community)
            bet = nx.betweenness_centrality(community)
            eig = nx.eigenvector_centrality(community)
            top_n_deg = nlargest(n, deg, key=deg.get)
            top_n_bet = nlargest(n, bet, key=bet.get)
            top_n_eig = nlargest(n, eig, key=eig.get)
            return list({*top_n_deg,*top_n_bet,
            *top_n_eig
            })

        

        for i in range(len(self.homologue_communities)):
            self.central_nodes.append(find_central_nodes(self.homologue_communities[i]))


    def set_c_nodes_neighbour(self):
        def find_c_nodes_neighbour(community, n=3):
            if len(community) < Network.MIN_SIZE: return []
            deg = nx.degree_centrality(community)
            bet = nx.betweenness_centrality(community)

            top_n_deg = nlargest(n, deg, key=deg.get)
            top_n_bet = nlargest(n, bet, key=bet.get)

            return list({*top_n_deg, 
            *top_n_bet
            })

        for i in range(len(self.adjacent_communities)):
            neigh_networks = map(self.graph.subgraph, self.adjacent_communities[i])
            cen_neigh = map(find_c_nodes_neighbour, neigh_networks)
            self.central_nodes_neighbour.append(cen_neigh)

    def node_info(self, node, lst):
        spath = nx.shortest_path(self.graph, source=self.homologue, target=node)
        return {
            "times_occurred": lst.count(node),
            "distance": len(spath)
        }

    def set_important_nodes(self):
        # flatten the central nodes list
        flat_central_nodes = sum(self.central_nodes,[])
        for node in set(flat_central_nodes):
            self.important_nodes[node] = self.node_info(node, flat_central_nodes)

    def set_important_nodes_neighbour(self):
        # flatten the central nodes list
        flat_central_nodes_1 = sum( self.central_nodes_neighbou,[])
        flat_central_nodes_2 = sum( flat_central_nodes_1 ,[])
        for node in set(flat_central_nodes_2):
            self.important_nodes_neighbour[node] = self.node_info(node, flat_central_nodes_2)

    def find_neighbours(self):
        for comm in self.homologue_communities:
            nodes = comm.nodes
            neighs = set()
            for n in nodes:
                neighs.update([*self.graph.neighbors(n)])
            self.community_neighbours.append(neighs)

    def set_neighbour_communities(self):
        a = self.partitions.copy()
        for i, part in enumerate(a):
            del part[self.homologue_index[i]]
            neighs = self.community_neighbours[i]
            # all communities containing a neighbouring element
            nei_comm = [comm for comm in part if set(comm) & set(neighs) != set()]
            self.adjacent_communities.append(nei_comm)

    @property
    def partitions(self):
        return self._partitions
    
    # def get_partitions(self):
    #     return self.partitions

    def get_homologue_communities(self):
        return self.homologue_communities

    def get_central_nodes(self):
        return self.central_nodes
    
    def get_important_nodes(self):
        return self.important_nodes

In [8]:
akt2 = Network(G, '4932.YKL126W')

In [9]:
index_list=enumerate(G)
num_to_node={i[0]:i[1] for i in index_list}
index_list=enumerate(G)
node_to_num={i[1]:i[0] for i in index_list}

In [10]:
def dissimilarity_matrix(partitions):
    N=len(partitions)
    index_part=[]
    for part in partitions:
        idx=[[node_to_num[n] for n in suba] for suba in part]
        index_part.append(idx)
    
    combs_idx=[]
    for part in index_part:
        kk=[combinations(x,2) for x in part]
        combs_idx+=kk

    aa=sum([*map(list,combs_idx)],[])
    aa2=map(frozenset,aa)
    cc=Counter(aa2)

    sim_val=cc.values()
    dis_val=np.array([*sim_val])


    coord=[[*i] for i in cc.keys()]
    coord_mat=np.array(coord)

    row=np.concatenate((coord_mat[:,1],coord_mat[:,0]))
    col=np.concatenate((coord_mat[:,0],coord_mat[:,1]))

    dim=len(G.nodes)
    dist=np.concatenate((dis_val,dis_val))
    sim_mat=ss.coo_matrix((dist, (row, col)), shape=(dim,dim))

    sim_arr=sim_mat.toarray()
    dis_arr=1-sim_arr/N
    np.fill_diagonal(dis_arr, 0)

    return dis_arr

In [11]:
dist_mat=dissimilarity_matrix(akt2.partitions)

In [12]:
F=G.copy()
F=nx.relabel_nodes(F,node_to_num)

In [13]:
#Import the command for hierarchical clustering
from sklearn.cluster import AgglomerativeClustering

In [14]:
lnes=[*map(len,akt2.partitions)]

In [15]:
hier_parts_complete=[]
hier_parts_avg=[]
for i in range(min(lnes),max(lnes)+1):
    c_mod=AgglomerativeClustering(n_clusters = i, affinity="precomputed",linkage="complete")
    hh=c_mod.fit_predict(dist_mat)
    hier_parts_complete.append(hh)
    c_moda=AgglomerativeClustering(n_clusters = i, affinity="precomputed",linkage="average")
    hha=c_moda.fit_predict(dist_mat)
    hier_parts_avg.append(hha)

In [16]:
nodesF=np.array(list(F.nodes))
def part_collector(hier):
    m=max(hier)
    coll=[set(nodesF[hier==i].tolist()) for i in range(m+1)]
    return coll

In [17]:
fpartC=[*map(part_collector,hier_parts_complete)]
fpartA=[*map(part_collector,hier_parts_avg)]

In [18]:
C_mod=np.zeros(max(lnes)+1-min(lnes))
A_mod=np.zeros(max(lnes)+1-min(lnes))
for i,part in enumerate(fpartC):
    modul=nx_comm.modularity(F,part)
    C_mod[i]=(modul)

In [19]:
for i,part in enumerate(fpartA):
    modul=nx_comm.modularity(F,part)
    A_mod[i]=(modul)

In [20]:
print(A_mod.max())
idxA=np.where(A_mod==A_mod.max())[0][0]

0.6796802540971084


In [21]:
intr=fpartA[idxA].copy()

In [22]:
ypk1_comm=[num_to_node[k] for k in intr[4]]

In [23]:
g1=G.subgraph(ypk1_comm)

In [24]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx

In [25]:
#Choose a title!
title = 'Protein Network'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [("Protein", "@index")]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object with spring layout
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(g1, nx.spring_layout, scale=10, center=(0, 0))

#Set node size and color
network_graph.node_renderer.glyph = Circle(size=15, fill_color='skyblue')

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

#Add network graph to the plot
plot.renderers.append(network_graph)

show(plot)
#save(plot, filename=f"{title}.html")

In [26]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap

In [27]:
degrees = dict(nx.degree(g1))
nx.set_node_attributes(g1, name='degree', values=degrees)

In [31]:
number_to_adjust_by = 5
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in nx.degree(G)])
nx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)

In [35]:
#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'adjusted_node_size'

#Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = Blues8

#Choose a title!
title = 'Protein network'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
       ("Protein", "@index"),
        ("Degree", "@degree")
]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html\
network_graph = from_networkx(g1, nx.random_layout,  center=(0, 0))

#Set node sizes and colors according to node degree (color as spectrum of color palette)
minimum_value_color = min(network_graph.node_renderer.data_source.data[color_by_this_attribute])
maximum_value_color = max(network_graph.node_renderer.data_source.data[color_by_this_attribute])
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=linear_cmap(color_by_this_attribute, color_palette, minimum_value_color, maximum_value_color))

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

plot.renderers.append(network_graph)

show(plot)
#save(plot, filename=f"{title}.html")

In [67]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap

In [68]:
degrees = dict(nx.degree(F))
nx.set_node_attributes(F, name='degree', values=degrees)

In [69]:
number_to_adjust_by = 5
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in nx.degree(G)])
nx.set_node_attributes(F, name='adjusted_node_size', values=adjusted_node_size)

In [70]:
from bokeh.palettes import Turbo256

In [71]:
# Create empty dictionaries
modularity_class = {}
modularity_color = {}
#Loop through each community in the network
for community_number, community in enumerate(intr):
    #For each member of the community, add their community number and a distinct color
    for name in community: 
        modularity_class[name] = community_number
        modularity_color[name] = Turbo256[community_number*10]

In [72]:
# Add modularity class and color as attributes from the network above
nx.set_node_attributes(F, modularity_class, 'modularity_class')
nx.set_node_attributes(F, modularity_color, 'modularity_color')

In [73]:
#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'modularity_color'
#Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = Blues8
#Choose a title!
title = 'Game of Thrones Network'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
       ("Character", "@index"),
        ("Degree", "@degree"),
         ("Modularity Class", "@modularity_class"),
        ("Modularity Color", "$color[swatch]:modularity_color"),
]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset, tap", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(F, nx.spring_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as category from attribute)
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=color_by_this_attribute)

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

plot.renderers.append(network_graph)

show(plot)
#save(plot, filename=f"{title}.html")

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "size" value "adjusted_node_size" [renderer: GlyphRenderer(id='3813', ...)]
